<a href="https://colab.research.google.com/github/2kunal6/code/blob/master/ml/deep_learning/pytorch/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import glob 
import zipfile
import numpy as np
import cv2
import os
import time

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

import pandas as pd
from sklearn.model_selection import train_test_split


data_root_path = 'drive/My Drive/colab_data/dogs-vs-cats/'
test_path = data_root_path + 'data/test1'
train_path = data_root_path + 'data/train'

def unzip_files():
    zip_files = glob.glob(data_root_path + '/*.zip')
    for zip_file in tqdm(zip_files):
        with zipfile.ZipFile(zip_file, 'r') as zf:
            zf.extractall(data_root_path + "data")


IMAGE_SIZE = (3, 50, 50)
train_data = []

def make_training_data():
    if(not os.path.exists(test_path)):
      unzip_files()
    LABELS = {'cat': 0, 'dog': 1}
    for f in tqdm(os.listdir(train_path)):
        path = os.path.join(train_path, f)
        if("jpg" not in f):
            continue
        img=cv2.imread(path)
        #img=cv2.resize(img, IMAGE_SIZE)
        train_data.append([img, np.eye(len(LABELS))[LABELS[f.split('.')[0]]]])
    
    np.random.shuffle(train_data)
    np.save(data_root_path + "color_train_data.npy", train_data)

if(not os.path.exists(data_root_path + "color_train_data.npy")):
    make_training_data()

train_data = np.load(data_root_path + "color_train_data.npy", allow_pickle=True)
np.random.shuffle(train_data)
print(train_data[0])

OSError: ignored

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
  

all_X = torch.Tensor([i[0] for i in train_data]).view(-1, IMAGE_SIZE)
all_X = all_X/255.0
all_y = torch.Tensor([i[1] for i in train_data])

X_train, X_test, y_train, y_test = train_test_split(all_X, all_y, test_size=0.1, stratify=all_y)


net = models.resnet18(pretrained=True).to(device)
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
net.fc = nn.Linear(net.fc.in_features, 2)


def run(X, y, is_training=True):
    
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    
    if is_training:
        net.zero_grad()
        optimizer.zero_grad()
        
    outputs = net(X)
    matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)

    if is_training:
        loss.backward()
        optimizer.step()

    return acc, loss

def batch_and_run(X, y, is_training=True):
    BATCH_SIZE = 64
    for i in tqdm(range(0, len(X), BATCH_SIZE)): 
        batch_X = X[i:i+BATCH_SIZE].view(-1, IMAGE_SIZE).to(device)
        batch_y = y[i:i+BATCH_SIZE].to(device)
            
        return run(batch_X, batch_y, is_training)


def train(net):
    EPOCHS = 20  
    for epoch in range(EPOCHS):
        acc, loss = batch_and_run(X_train, y_train, True)
        val_acc, val_loss = batch_and_run(X_test, y_test, False)
        print(f"Epoch: {epoch}, {round(time.time(),3)},{round(float(acc),2)},{round(float(loss), 4)},{round(float(val_acc),2)},{round(float(val_loss),4)}\n")
        
train(net)


predictions=[]
id_line=[]
def test(net):
    for f in os.listdir(test_path):
        id_line.append(f.split('.')[0])
        path = os.path.join(test_path, f)
        img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img, (50, 50))
        test_X = torch.Tensor(img).view(-1, IMAGE_SIZE)
        test_X = test_X.to(device)
        net_out = net(test_X)
        
        predictions.append(torch.argmax(net_out))

test(net)

predicted_val = [x.tolist() for x in predictions]
print(predicted_val)

Running on the GPU


AttributeError: ignored